# Bot de Agendamento de Consultas Médicas com Memória de Curto Prazo

Este notebook adapta o bot originalmente desenvolvido para o projeto **MikuSec** para que funcione como um assistente de agendamento de consultas em uma clínica médica.  O bot coleta dados básicos do paciente (nome completo, data de nascimento, CPF, RG, endereço e plano de saúde) por meio de uma conversa sequencial no Telegram.  Ao final da coleta, as informações são armazenadas em uma ontologia simples usando a biblioteca **Owlready2** e o bot confirma os dados com o usuário.


## Instruções para uso

1. **Criar o bot no Telegram:** use o BotFather (`/newbot`) para gerar o token de acesso.  Guarde esse token com segurança.
2. **Instalar dependências:** execute a célula de instalação para instalar a biblioteca `owlready2` (necessária para manipular ontologias).
3. **Configurar o token:** insira o token do seu bot na variável `TOKEN` na célula de configuração.
4. **Executar as células:** após configurar o token, execute todas as células restantes.  A última célula inicia o laço principal que escuta novas mensagens e conduz a coleta de dados.
5. **Encerrar a execução:** para parar o bot, interrompa a execução da célula do laço principal.


In [ ]:
# Instala a biblioteca Owlready2 (necessária para manipular ontologias)
!pip -q install owlready2


In [ ]:
from owlready2 import *

# Cria ou carrega uma ontologia simples para armazenar dados de pacientes
onto = get_ontology("http://example.org/clinic.owl")

with onto:
    class Paciente(Thing):
        pass
    class nome(DataProperty):
        domain = [Paciente]
        range = [str]
    class dataNascimento(DataProperty):
        domain = [Paciente]
        range = [str]
    class cpf(DataProperty):
        domain = [Paciente]
        range = [str]
    class rg(DataProperty):
        domain = [Paciente]
        range = [str]
    class endereco(DataProperty):
        domain = [Paciente]
        range = [str]
    class planoSaude(DataProperty):
        domain = [Paciente]
        range = [str]

# Salva a ontologia em disco; você pode reusar esse arquivo posteriormente
onto.save(file="clinic.owl")
print("Ontologia criada e salva como clinic.owl")


In [ ]:
import json
import requests
import time
import urllib

# Carrega a ontologia previamente definida (clinic.owl)
from owlready2 import get_ontology, sync_reasoner_pellet
onto = get_ontology("clinic.owl").load()

# Insira aqui o token do seu bot do Telegram (obtido via BotFather)
TOKEN = ''  # <-- Substitua pelo token do seu bot

if not TOKEN:
    raise ValueError('Você precisa inserir o token do seu bot na variável TOKEN para continuar.')

URL = f'https://api.telegram.org/bot{TOKEN}/'

# Variáveis de estado e armazenamento de dados dos pacientes
estado = {}        # controla em que passo da conversa o usuário está
paciente_info = {} # armazena temporariamente as informações coletadas de cada usuário


In [ ]:
def get_url(url: str) -> str:
    """Efetua uma requisição GET e devolve o conteúdo como string."""
    response = requests.get(url)
    return response.content.decode('utf8')


def get_json_from_url(url: str) -> dict:
    """Efetua uma requisição GET e devolve o conteúdo JSON."""
    return json.loads(get_url(url))


def get_updates(offset: int = None) -> dict:
    """Obtém as atualizações (mensagens) do Telegram a partir de um offset."""
    url = URL + 'getUpdates'
    if offset:
        url += f'?offset={offset}'
    return get_json_from_url(url)


def get_last_update_id(updates: dict) -> int:
    """Recupera o ID da última atualização recebida."""
    update_ids = [int(update['update_id']) for update in updates['result']]
    return max(update_ids)


def send_message(text: str, chat_id: int) -> None:
    """Envia uma mensagem de texto para o usuário."""
    encoded_text = urllib.parse.quote_plus(text)
    url = URL + f'sendMessage?text={encoded_text}&chat_id={chat_id}'
    get_url(url)


def process_updates(updates: dict) -> None:
    """Processa cada nova mensagem e gerencia o fluxo de coleta de dados."""
    for update in updates['result']:
        if 'message' not in update or 'text' not in update['message']:
            continue  # ignora mensagens que não contêm texto
        chat_id = update['message']['chat']['id']
        text = update['message']['text']

        # Inicializa o estado e o dicionário de dados para novos usuários
        if chat_id not in estado:
            estado[chat_id] = 0
            paciente_info[chat_id] = {}

        step = estado[chat_id]

        # Passo 0: saudação e solicitação do nome
        if step == 0:
            send_message('Olá! Vamos agendar sua consulta. Qual é o seu nome completo?', chat_id)
            estado[chat_id] = 1
            continue

        # Passo 1: coleta do nome
        if step == 1:
            paciente_info[chat_id]['nome'] = text
            send_message('Qual é a sua data de nascimento? (dd/mm/aaaa)', chat_id)
            estado[chat_id] = 2
            continue

        # Passo 2: coleta da data de nascimento
        if step == 2:
            paciente_info[chat_id]['dataNascimento'] = text
            send_message('Informe seu CPF:', chat_id)
            estado[chat_id] = 3
            continue

        # Passo 3: coleta do CPF
        if step == 3:
            paciente_info[chat_id]['cpf'] = text
            send_message('Informe seu RG:', chat_id)
            estado[chat_id] = 4
            continue

        # Passo 4: coleta do RG
        if step == 4:
            paciente_info[chat_id]['rg'] = text
            send_message('Qual é o seu endereço completo?', chat_id)
            estado[chat_id] = 5
            continue

        # Passo 5: coleta do endereço
        if step == 5:
            paciente_info[chat_id]['endereco'] = text
            send_message('Qual é o nome do seu plano de saúde?', chat_id)
            estado[chat_id] = 6
            continue

        # Passo 6: coleta do plano de saúde e finalização
        if step == 6:
            paciente_info[chat_id]['planoSaude'] = text

            # Cria um novo indivíduo da classe Paciente e grava os dados na ontologia
            with onto:
                p = Paciente()
                p.nome = [paciente_info[chat_id]['nome']]
                p.dataNascimento = [paciente_info[chat_id]['dataNascimento']]
                p.cpf = [paciente_info[chat_id]['cpf']]
                p.rg = [paciente_info[chat_id]['rg']]
                p.endereco = [paciente_info[chat_id]['endereco']]
                p.planoSaude = [paciente_info[chat_id]['planoSaude']]
            onto.save(file='clinic.owl')

            # Monta mensagem de resumo
            resumo = (
                f"Obrigado, {paciente_info[chat_id]['nome']}!
"
                f"Dados cadastrados:
"
                f"• Data de nascimento: {paciente_info[chat_id]['dataNascimento']}
"
                f"• CPF: {paciente_info[chat_id]['cpf']}
"
                f"• RG: {paciente_info[chat_id]['rg']}
"
                f"• Endereço: {paciente_info[chat_id]['endereco']}
"
                f"• Plano de saúde: {paciente_info[chat_id]['planoSaude']}
"
                'Em breve entraremos em contato para confirmar a data e horário da sua consulta.'
            )
            send_message(resumo, chat_id)
            # Define um estado final para este usuário
            estado[chat_id] = -1
            continue

        # Se o estado for -1, significa que a coleta já foi concluída
        if step == -1:
            send_message('Suas informações já foram registradas. Se precisar reagendar, por favor reinicie a conversa.', chat_id)
            continue


In [ ]:
# Loop principal
last_update_id = None
print('Bot da clínica iniciado. Aguardando mensagens...')

while True:
    updates = get_updates(last_update_id)
    if len(updates.get('result', [])) > 0:
        last_update_id = get_last_update_id(updates) + 1
        process_updates(updates)
    time.sleep(1)
